### Stage 1: Data Feasibility Evaluation over GPC sites.
#### Objective 1.1: Study cohort extraction and characterization

##### Inclusion criteria:

* (IP,IS,EI) visits with length of stay (LOS) >= 2 days; 
* age at visit >= 18 years old
* had at least 2 SCr measurements during stay

***

##### Exclusion criteria:

* initial eGFR < 15 mL/min per 1.73m^2 at visit; 
* any dialysis procedure or renal transplantation prior to visit;
* received RRT within 48 hours since first SCr; 
* burn patients (DX)

***

##### AKI Staging

AKI Stages are defined based on [KDIGO](http://www.kdigo.org/clinical_practice_guidelines/pdf/KDIGO%20AKI%20Guideline.pdf):

* AKI 1: increase in serum creatinine by >=**0.3 mg/dL** within **48 hours** OR **1.5-1.9 times** baseline^1 within **7 days**; 
* AKI 2: **2.0 to 2.9 times** baseline within **7 days**; 
* AKI 3: increase in serum creatinine to >= **4.0 mg/dL** within **48 hours** OR **3.0 times** baseline^1 within **7 days** OR **initiation of RRT**          

*baseline is defined as initial SCr during hospitalization    

***

##### Implementation: Extracting AKI Study Cohort

In [ ]:
#markdown file printout setup
knitr::opts_chunk$set(message=F,warning=F)

#source utility functions
#note: need to change "./" to "../" if directly knit this report
source("util.R")

#load libraries
require_libraries(c("DBI",
                    "tidyr",
                    "dplyr",
                    "magrittr",
                    "stringr",
                    "knitr",
                    "kableExtra",
                    "ggplot2",
                    "ggrepel",
                    "RCurl",
                    "XML",
                    "openxlsx",
                    "diagram",
                    "janitor"),verb=F)

In [ ]:
site<-'KUMC'
dataFolder<-paste0("/home/hchan2/AKI/data/",site,"/raw/")
targetFolder<-paste0("/home/hchan2/AKI/data/",site,"/")

In [ ]:
Table1<-janitor::clean_names(read.csv(paste0(dataFolder,"AKI_ONSETS",".csv")), case='all_cap')
consort_tbl<-janitor::clean_names(read.csv(paste0(dataFolder,"CONSORT_DIAGRAM_BC",".csv")), case='all_cap')
#Table1<-janitor::clean_names(readRDS(paste0(dataFolder,"AKI_ONSETS","_POSIX",".rda")), case='all_cap')
#consort_tbl<-janitor::clean_names(readRDS(paste0(dataFolder,"CONSORT_DIAGRAM",".rda")), case='all_cap')

In [ ]:
# collect summaries
enc_tot<-n_distinct(Table1$ENCOUNTERID)

tbl1_dsa<-Table1 %>% 
  dplyr::select(PATID,ENCOUNTERID,
                NONAKI_SINCE_ADMIT, 
                AKI1_SINCE_ADMIT,
                AKI2_SINCE_ADMIT,
                AKI3_SINCE_ADMIT) %>%
  gather(stage, days_since_admit,-PATID,-ENCOUNTERID) %>%
  mutate(stage=gsub("_.*","",stage)) %>% 
  filter(!is.na(days_since_admit)) 

tbl1_summ<-tbl1_dsa %>%
  group_by(stage) %>%
  dplyr::summarize(pat_cnt=n_distinct(PATID),
                   enc_cnt=n_distinct(ENCOUNTERID),
                   min_time=min(days_since_admit,na.rm=T),
                   q1_time=quantile(days_since_admit,probs=0.25,na.rm=T),
                   median_time=median(days_since_admit,na.rm=T),
                   mean_time=round(mean(days_since_admit,na.rm=T),1),
                   q3_time=quantile(days_since_admit,probs=0.75,na.rm=T),
                   max_time=max(days_since_admit,na.rm=T),
                   sd_time=round(sd(days_since_admit,na.rm=T),2)) %>%
  mutate(semi_IQR_time=0.5*(q3_time-q1_time)) %>%
  #HIPAA, low counts masking
  mutate(pat_cnt=ifelse(as.numeric(pat_cnt)<11,"<11",as.character(pat_cnt)),
         enc_cnt=ifelse(as.numeric(enc_cnt)<11,"<11",as.character(enc_cnt)))

#save results
saveRDS(Table1,file=paste0(targetFolder,"Table1","_POSIX",".rda"))
saveRDS(consort_tbl,file=paste0(targetFolder,"consort_tbl","_POSIX",".rda"))

#clean up
#rm(cohort); gc()

In [ ]:
#consort diagram
#if("Less_than_2_SCr" %in% consort_tbl$CNT_TYPE)
#{
#  tempr<-which(grepl("Has_at_least_1_SCr", consort_tbl$CNT_TYPE))
#  consort_tbl[tempr]$ENC_CNT <- consort_tbl$ENC_CNT[which(grepl("Has_at_least_1_SCr", consort_tbl$CNT_TYPE))] - 
#                                            consort_tbl$ENC_CNT[which(grepl("Less_than_2_SCr", consort_tbl$CNT_TYPE))]
#  consort_tbl[tempr]$CNT_TYPE <- "Has_at_least_2_SCr"
#}
consort_diag(consort_tbl)

In [ ]:
# auxilliary summaries and tables
enc_tot<-n_distinct(Table1$ENCOUNTERID)
# critical dates of AKI encounters
aki_stage_ind<-Table1 %>%
  dplyr::select(ENCOUNTERID, ADMIT_YEAR,
                NONAKI_SINCE_ADMIT, AKI1_SINCE_ADMIT,AKI2_SINCE_ADMIT,AKI3_SINCE_ADMIT) %>%
  gather(chk_pt, critical_date,-ENCOUNTERID) %>%
  filter(!is.na(critical_date)) %>%
  mutate(chk_pt=gsub("_.*","",chk_pt)) %>%
  group_by(chk_pt) %>%
  dplyr::mutate(stg_tot_cnt=n()) %>%
  ungroup %>%
  arrange(ENCOUNTERID, chk_pt, critical_date, stg_tot_cnt)

***

#### Objective 1.2: Variable Collection and Summaries (Table by Table)
In this section, we will collect variables from PCORNET_CDM tables: *DEMOGRAPHIC*, *ENCOUNTER*, *VITAL*, *LAB_RESULT_CM*, *PRESCRIBING MEDICATION*, *DIAGNOSIS*, *PROCEDURE*, check data quality and generate variable summaries table by table.

##### Demographic 
Demographic variables include *Age (calculated from Birth_Date)*, *Sex*, *Race*, *Ethnicity*.  

In [ ]:
#send completed sql script over to database via established connection
demo<-janitor::clean_names(read.csv(paste0(dataFolder,"AKI_DEMO",".csv")), case='all_cap') %>% dplyr::rename(ENCOUNTERID=ONSETS_ENCOUNTERID)

In [ ]:
#perform some preliminary data cleaning and covert to long skinny format
demo %<>%
  mutate(AGE_GRP=case_when(AGE<= 25 ~ "18-25",
                           AGE >= 26 & AGE <= 35 ~ "26-35",
                           AGE >= 36 & AGE <= 45 ~ "36-45",
                           AGE >= 46 & AGE <= 55 ~ "46-55",
                           AGE >= 56 & AGE <= 65 ~ "56-65",
                           AGE >= 66 ~ "66<=")) %>%
  dplyr::select(ENCOUNTERID,
                AGE,AGE_GRP,SEX,RACE,HISPANIC,DDAYS_SINCE_ENC) %>%
  replace_na(list(AGE="NI",
                  AGE_GRP="NI",
                  SEX="NI",
                  RACE="NI",
                  HISPANIC="NI")) %>%
  gather(key,value,-ENCOUNTERID) %>%
  distinct()


#collect summaries
demo_summ<-aki_stage_ind %>% 
  dplyr::filter(!chk_pt %in% c("DISCHARGE")) %>%
  dplyr::select(-critical_date) %>%
  left_join(demo %>% 
              dplyr::filter(!(key %in% c("AGE","DDAYS_SINCE_ENC"))), 
            by="ENCOUNTERID") %>%
  group_by(chk_pt,stg_tot_cnt,key,value) %>%
  #HIPAA compliance, low count masking
  dplyr::summarize(enc_cnt = ifelse(n()<11,11,n())) %>%
  mutate(enc_prop = ifelse(enc_cnt>11,round(enc_cnt/stg_tot_cnt[1],3),11)) %>%
  ungroup %>%
  dplyr::select(-stg_tot_cnt) %>%
  gather(summ,summ_val,-chk_pt,-key,-value) %>%
  bind_rows(aki_stage_ind %>%
              dplyr::filter(!chk_pt %in% c("DISCHARGE")) %>%
              dplyr::select(chk_pt,stg_tot_cnt) %>% 
              distinct() %>%
              #HIPAA compliance, low count masking
              dplyr::rename(enc_cnt=stg_tot_cnt) %>%
              mutate(enc_cnt=ifelse(enc_cnt<11,11,enc_cnt)) %>%
              mutate(enc_prop=ifelse(enc_cnt>11,round(enc_cnt/enc_tot,3),11),
                     key="TOTAL",
                     value="(%/overall)") %>%
              gather(summ,summ_val,-chk_pt,-key,-value) %>%
              dplyr::select(key,value,chk_pt,summ,summ_val)) %>%
  unite("stg_summ",c("chk_pt","summ")) %>%
  distinct() %>% spread(stg_summ,summ_val) %>%
  replace(.,is.na(.),0)

#save results
#saveRDS(demo,file=paste0(targetFolder,"DEMO","_POSIX",".rda"))
#saveRDS(demo_summ,file=paste0(targetFolder,"demo_summ","_POSIX",".rda"))

#clean up
rm(demo); gc()

In [ ]:
demo_nice_tbl<-demo_summ %>%
  gather(summ,summ_val,-key,-value) %>%
  mutate(summ_val=ifelse(grepl("_prop",summ),summ_val*100,summ_val)) %>%
  mutate(summ_val=as.character(summ_val)) %>%
  mutate(summ_val=ifelse(grepl("_enc",summ) & summ_val=="11","<11",summ_val)) %>%
  mutate(summ_val=ifelse(grepl("_prop",summ) & summ_val=="1100","<11",
                         ifelse(grepl("_prop",summ) & summ_val!="1100",paste0(summ_val,"%"),summ_val))) %>%
  spread(summ,summ_val) %>%
  unite("ADMIT",paste0("ADMIT_",c("enc_cnt","enc_prop")),sep=", ") %>%
  unite("AKI1",paste0("AKI1_",c("enc_cnt","enc_prop")),sep=", ") %>%
  unite("AKI2",paste0("AKI2_",c("enc_cnt","enc_prop")),sep=", ") %>%
  unite("AKI3",paste0("AKI3_",c("enc_cnt","enc_prop")),sep=", ") %>%
  unite("NONAKI",paste0("NONAKI_",c("enc_cnt","enc_prop")),sep=", ") %>%
  arrange(key,value)

row_grp_pos<-demo_nice_tbl %>% 
  mutate(rn=1:n()) %>%
  group_by(key) %>%
  dplyr::summarize(begin=rn[1],
                   end=rn[n()]) %>%
  ungroup

# kable(demo_nice_tbl,
#       caption="Table1 - Demographic Summaries at AKI1, AKI2, AKI3") %>%
#   kable_styling("striped", full_width = F) %>%
#   group_rows("Age Group", row_grp_pos$begin[1],row_grp_pos$end[1]) %>%
#   group_rows("Hispanic", row_grp_pos$begin[2],row_grp_pos$end[2]) %>%
#   group_rows("Race", row_grp_pos$begin[3],row_grp_pos$end[3]) %>%
#   group_rows("Sex", row_grp_pos$begin[4],row_grp_pos$end[4]) %>%  
#   group_rows("Total",row_grp_pos$begin[5],row_grp_pos$end[5])  
demo_nice_tbl

Demographic characterizations for patients at different AKI stages are summarized in Table 1.

***

##### Vital 
Vital variables include: *Height*, *Weight*, *BMI*, *Blood Pressure (Systolic, Diastolic)*, *Smoking Status*. 

In [ ]:
## vital
vital<-janitor::clean_names(read.csv(paste0(dataFolder,"AKI_VITAL",".csv")), case='all_cap') %>% 
#dplyr::rename(ENCOUNTERID=ONSETS_ENCOUNTERID) %>%
  mutate(BMI_GRP = case_when(ORIGINAL_BMI <= 25 ~ "BMI <= 25",
                             ORIGINAL_BMI > 25 &  ORIGINAL_BMI <= 30 ~ "BMI 26-30",
                             ORIGINAL_BMI >=31  ~ "BMI >= 31")) %>%
  dplyr::rename(dsa=DAYS_SINCE_ADMIT,
                timestamp=MEASURE_DATE_TIME) %>%
  gather(key,value,-PATID,-ENCOUNTERID,-dsa,-timestamp) %>%
  dplyr::filter(!is.na(key) & !is.na(value)) %>%
  mutate(key=recode(key,
                    ORIGINAL_BMI="BMI",
                    SYSTOLIC="BP_SYSTOLIC",
                    DIASTOLIC="BP_DIASTOLIC")) %>%
  distinct()


vital1<-vital %>%
  dplyr::select(ENCOUNTERID, key, value, dsa) %>%
  dplyr::filter(key %in% c("HT","WT","BMI","BP_DIASTOLIC","BP_SYSTOLIC")) %>%
  mutate(value=as.numeric(value)) %>%
  mutate(param_low=case_when(key=="HT" ~ 0,
                             key=="WT" ~ 0,
                             key=="BMI" ~ 0,
                             key %in% c("BP_DIASTOLIC",
                                        "BP_SYSTOLIC") ~ 40),
         param_high=case_when(key=="HT" ~ 94.99,
                              key=="WT" ~ 350,
                              key=="BMI" ~ 50,
                              key=="BP_DIASTOLIC"~120,
                              key=="BP_SYSTOLIC" ~ 210)) %>%
  mutate(dsa_grp=case_when(dsa < 0 ~ "0>",
                           dsa >=0 & dsa < 1 ~ "1",
                           dsa >=1 & dsa < 2 ~ "2",
                           dsa >=2 & dsa < 3 ~ "3",
                           dsa >=3 & dsa < 4 ~ "4",
                           dsa >=4 & dsa < 5 ~ "5",
                           dsa >=5 & dsa < 6 ~ "6",
                           dsa >=6 & dsa < 7 ~ "7",
                           dsa >=7 ~ "7<"))


# collect summaries
vital_summ<-vital1 %>%
  group_by(key) %>%
  dplyr::summarize(record_cnt=n(),
                   enc_cnt=n_distinct(ENCOUNTERID),
                   low_cnt=sum((value<param_low)),
                   high_cnt=sum((value>param_high)),
                   min=min(value,na.rm=T),
                   mean=round(mean(value,na.rm=T)),
                   sd=round(sd(value,na.rm=T)),
                   median=round(median(value,na.rm=T)),
                   max=max(value,na.rm=T)) %>%
  ungroup %>%
  mutate(cov=round(sd/mean,1)) %>%
  #HIPAA, low counts masking
  mutate(enc_cnt=ifelse(as.numeric(enc_cnt)<11,"<11",enc_cnt),
         record_cnt=ifelse(as.numeric(record_cnt)<11,"<11",record_cnt),
         low_cnt=ifelse(as.numeric(low_cnt)<11,"<11",as.character(low_cnt)),
         high_cnt=ifelse(as.numeric(high_cnt)<11,"<11",as.character(high_cnt))) %>%
  gather(summ,overall,-key) %>%
  mutate(summ=recode(summ,
                     enc_cnt="1.encounters#",
                     record_cnt="2.records#",
                     low_cnt="3.low_records#",
                     high_cnt="4.high_records#",
                     min="5a.min",
                     median="5b.median",
                     mean="5c.mean",
                     sd="5d.sd",
                     cov="5e.cov",
                     max="5f.max")) %>%
  left_join(
    vital1 %>%
      group_by(key,dsa_grp) %>%
      dplyr::summarize(record_cnt=n(),
                       enc_cnt=n_distinct(ENCOUNTERID),
                       low_cnt=sum((value<param_low)),
                       high_cnt=sum((value>param_high)),
                       min=min(value,na.rm=T),
                       mean=round(mean(value,na.rm=T)),
                       sd=round(sd(value,na.rm=T)),
                       median=round(median(value,na.rm=T)),
                       max=max(value,na.rm=T)) %>%
      ungroup %>%
      mutate(cov=round(sd/mean,1)) %>%
      #HIPAA, low counts masking
      mutate(enc_cnt=ifelse(as.numeric(enc_cnt)<11,"<11",enc_cnt),
             record_cnt=ifelse(as.numeric(record_cnt)<11,"<11",record_cnt),
             low_cnt=ifelse(as.numeric(low_cnt)<11,"<11",as.character(low_cnt)),
             high_cnt=ifelse(as.numeric(high_cnt)<11,"<11",as.character(high_cnt))) %>%
      gather(summ,summ_val,-key,-dsa_grp) %>%
      spread(dsa_grp,summ_val) %>%
      mutate(summ=recode(summ,
                         enc_cnt="1.encounters#",
                         record_cnt="2.records#",
                         low_cnt="3.low_records#",
                         high_cnt="4.high_records#",
                         min="5a.min",
                         median="5b.median",
                         mean="5c.mean",
                         sd="5d.sd",
                         cov="5e.cov",
                         max="5f.max")),
    by=c("key","summ")
  ) %>%
  arrange(key,summ) %>%
  gather(days_from_admit,summ_val,-key,-summ) %>% 
  spread(summ,summ_val)


vital_smoke_summ<-vital %>%
  dplyr::select(PATID,ENCOUNTERID, key, value) %>%
  dplyr::filter(key %in% c("SMOKING","TOBACCO","TOBACCO_TYPE"))

#save
#saveRDS(vital,file=paste0(targetFolder,"VITAL","_POSIX",".rda"))
#saveRDS(vital_summ,file=paste0(targetFolder,"vital_summ","_POSIX",".rda"))
#saveRDS(vital_smoke_summ,file=paste0(targetFolder,"vital_smoke_summ","_POSIX",".rda"))

#clean up
rm(vital,vital1); gc()

In [ ]:
days_from_admit_label<-rev(c("0>", "1", "2", "3", "4", "5", "6", "7", "7<", "overall"))
if (!("BMI" %in% unique(vital_summ$key))){
  for (dayslabel in days_from_admit_label){
    vital_summ %<>% dplyr::add_row(key="BMI", days_from_admit=dayslabel, .before=1)
  }
}
if (!("BP_DIASTOLIC" %in% unique(vital_summ$key))){
  for (dayslabel in days_from_admit_label){
    vital_summ %<>% dplyr::add_row(key="BP_DIASTOLIC", days_from_admit=dayslabel, .before=11)
  }
}
if (!("BP_SYSTOLIC" %in% unique(vital_summ$key))){
  for (dayslabel in days_from_admit_label){
    vital_summ %<>% dplyr::add_row(key="BP_SYSTOLIC", days_from_admit=dayslabel, .before=21)
  }
}
if (!("HT" %in% unique(vital_summ$key))){
  for (dayslabel in days_from_admit_label){
    vital_summ %<>% dplyr::add_row(key="HT", days_from_admit=dayslabel, .before=31)
  }
}
if (!("WT" %in% unique(vital_summ$key))){
  for (dayslabel in days_from_admit_label){
    vital_summ %<>% dplyr::add_row(key="WT", days_from_admit=dayslabel, .before=41)
  }
}
# kable(vital_summ %>% dplyr::select(-key),
#       caption="Table 2a - Vital (HT,WT,BMI,SBP,DBP) Summaries") %>%
#   kable_styling("striped", full_width = F) %>%
#   group_rows("BMI", 1,10) %>%
#   group_rows("BP_DIASTOLIC", 11, 20) %>%
#   group_rows("BP_SYSTOLIC", 21, 30) %>%
#   group_rows("HT", 31, 40) %>%  
#   group_rows("WT", 41, 50)  

vital_summ

Table 2a identifies extreme values of vitals for height, weight, BMI, and blood pressure, which may suggest systemic errors such as typos, and conversion mistakes. Here is the list of how *high* and *low* vitals are defined (adopted from CDM EDC report):

|vital type  |upper bound for low values  |lower bound for high values  |     
|:----------:|:--------------------------:|:---------------------------:|   
|HT(inches)  |0                           |95                           |   
|WT(lbs)     |0                           |350                          |   
|BMI         |0                           |50                           |    
|BP_DIASTOLIC|40                          |120                          |     
|BP_SYSTOLIC |40                          |210                          |     


In [ ]:
vital_smoke_summ %<>%
  distinct() %>%
  group_by(PATID,ENCOUNTERID, key) %>%
  dplyr::mutate(value=paste(value[order(value)],collapse = ",")) %>% 
  ungroup %>% distinct() %>%
  spread(key,value) %>%
  right_join(Table1 %>% dplyr::select(PATID,ENCOUNTERID),
             by=c("PATID","ENCOUNTERID")) %>%
  replace_na(list(SMOKING="NI",
                  TOBACCO="NI",
                  TOBACCO_TYPE="NI")) %>%
  gather(key,value,-PATID,-ENCOUNTERID) %>%
  mutate(key2=key) %>%
  unite("key_cat",c("key2","value")) %>%
  group_by(key,key_cat) %>%
  dplyr::summarize(pat_cnt=n_distinct(PATID),
                   enc_cnt=n_distinct(ENCOUNTERID),
                   enc_prop=n_distinct(ENCOUNTERID)/enc_tot) %>%
  arrange(desc(pat_cnt)) %>%
  ungroup %>%
  #HIPAA, low counts masking
  mutate(pat_cnt=ifelse(as.numeric(pat_cnt)<11,"<11",as.character(pat_cnt)),
         enc_cnt=ifelse(as.numeric(enc_cnt)<11,"<11",as.character(enc_cnt))) %>%
  mutate(enc_prop=ifelse(enc_cnt!="<11",paste0(round(enc_prop,3)*100,"%"),"<11")) %>%
  gather(summ,summ_val,-key_cat,-key) %>%
  mutate(summ=recode(summ,
                     pat_cnt="1.patients#",
                     enc_cnt="2.encounters#",
                     enc_prop="3.encounters%")) %>%
  spread(summ,summ_val)


vital_smoke_summ %<>% dplyr::filter(`2.encounters#`!="<11") %>%
  arrange(key,desc(`2.encounters#`))

row_grp_pos<-vital_smoke_summ %>% 
  mutate(rn=1:n()) %>%
  group_by(key) %>%
  dplyr::summarize(begin=rn[1],
                   end=rn[n()]) %>%
  ungroup

# kable(vital_smoke_summ %>% dplyr::select(-key),
#       caption="Table 2b - Vital (SMOKING, TABACCO) Summaries") %>%
#   kable_styling("striped", full_width = F) %>%
#   group_rows(row_grp_pos$key[1], row_grp_pos$begin[1],row_grp_pos$end[1]) %>%
#   group_rows(row_grp_pos$key[2], row_grp_pos$begin[2],row_grp_pos$end[2]) %>%
#   group_rows(row_grp_pos$key[3], row_grp_pos$begin[3],row_grp_pos$end[3])

description<-"Table 2b identifies unreliable reporting of smoking status. A significant number of conflicting status or mismatch between smoking and tabacco summaries requires some further investigation."
vital_smoke_summ

In [ ]:
description<-"SMOKING, TABACCO, and TABACCO_TYPE columns are not populated"

`r description`

***

##### Labs

In [ ]:
## labs
lab<-janitor::clean_names(read.csv(paste0(dataFolder,"AKI_LAB",".csv")), case='all_cap') %>% 
dplyr::rename(ENCOUNTERID=ONSETS_ENCOUNTERID) %>%
  dplyr::rename(key=LAB_LOINC,value=RESULT_NUM,unit=RESULT_UNIT,
                dsa=DAYS_SINCE_ADMIT,timestamp=SPECIMEN_DATE_TIME) %>%
  dplyr::select(ENCOUNTERID,key,value,unit,dsa,timestamp) %>%
  dplyr::filter(!is.na(key) & !is.na(value)) %>%
  distinct() %>%
  mutate(dsa_grp=case_when(dsa < 0 ~ "0>",
                           dsa >=0 & dsa < 1 ~ "1",
                           dsa >=1 & dsa < 2 ~ "2",
                           dsa >=2 & dsa < 3 ~ "3",
                           dsa >=3 & dsa < 4 ~ "4",
                           dsa >=4 & dsa < 5 ~ "5",
                           dsa >=5 & dsa < 6 ~ "6",
                           dsa >=6 & dsa < 7 ~ "7",
                           dsa >=7 ~ "7<"))

#collect summaries
lab_summ<-lab %>% 
  group_by(key) %>%
  dplyr::summarize(record_cnt=n(),
                   enc_cnt=n_distinct(ENCOUNTERID),
                   min=min(value,na.rm=T),
                   mean=round(mean(value,na.rm=T),2),
                   sd=round(sd(value,na.rm=T),3),
                   median=round(median(value,na.rm=T)),
                   max=max(value,na.rm=T)) %>%
  ungroup %>%
  mutate(cov=round(sd/mean,3)) %>%
  mutate(freq_rk=rank(-enc_cnt,ties.method="first")) %>%
  #HIPAA, low counts masking
  mutate(enc_cnt=ifelse(as.numeric(enc_cnt)<11 & as.numeric(enc_cnt)>0,"<11",as.character(enc_cnt)),
         record_cnt=ifelse(as.numeric(record_cnt)<11 & as.numeric(record_cnt)>0,"<11",as.character(record_cnt))) %>%
  gather(summ,overall,-key,-freq_rk) %>%
  left_join(
    lab %>%
      group_by(key,dsa_grp) %>%
      dplyr::summarize(record_cnt=n(),
                       enc_cnt=n_distinct(ENCOUNTERID),
                       min=min(value,na.rm=T),
                       mean=round(mean(value,na.rm=T),2),
                       sd=round(sd(value,na.rm=T),3),
                       median=round(median(value,na.rm=T)),
                       max=max(value,na.rm=T)) %>%
      ungroup %>%
      mutate(cov=round(sd/mean,3)) %>%
      #HIPAA, low counts masking
      mutate(enc_cnt=ifelse(as.numeric(enc_cnt)<11 & as.numeric(enc_cnt)>0,"<11",as.character(enc_cnt)),
             record_cnt=ifelse(as.numeric(record_cnt)<11 & as.numeric(record_cnt)>0,"<11",as.character(record_cnt)),
             sd=ifelse(is.nan(sd),0,sd)) %>%
      gather(summ,summ_val,-key,-dsa_grp) %>%
      spread(dsa_grp,summ_val),
    by=c("key","summ")
  ) %>%
  arrange(freq_rk,summ) %>%
  #additional 
  mutate(at_admission=ifelse(is.na(`1`),0,1))

#save
#saveRDS(lab,file=paste0(targetFolder,"LAB","_POSIX",".rda"))
#saveRDS(lab_summ,file=paste0(targetFolder,"lab_summ","_POSIX",".rda"))

#clean up
rm(lab); gc()

In [ ]:
lab_temp<-lab_summ %>%
  dplyr::filter(summ %in% c("enc_cnt","record_cnt")) %>%
  dplyr::select(key,summ,overall) %>% distinct() %>%
  dplyr::filter(overall!="<11") %>%
  mutate(overall=as.numeric(overall)) %>%
  spread(summ,overall,fill=0) %>%
  dplyr::filter(enc_cnt>=enc_tot*0.01) %>%
  mutate(record_intensity=round(record_cnt/enc_cnt,2)) %>%
  mutate(label=ifelse(dense_rank(-enc_cnt)<=10 | dense_rank(-record_intensity)<=10,as.character(key),""))

ggplot(lab_temp,aes(x=record_intensity,y=enc_cnt,label=label))+
  geom_point()+ geom_text_repel(segment.alpha=0.5,segment.color="grey")+
  scale_y_continuous(sec.axis = sec_axis(trans= ~./enc_tot,
                                         name = 'Percentage'))+
  labs(x="Average Records per Encounter",
       y="Encounter Counts",
       title="Figure 1 - Data Density vs. Records Intensity")

#get their searchable link
lab_report<-lab_temp %>%
  dplyr::filter(key != "NI") %>%
  arrange(desc(enc_cnt)) %>% 
  dplyr::select(key) %>%
  distinct() %>% dplyr::slice(1:5) %>%
  bind_rows(
    lab_temp %>% 
      dplyr::filter(key != "NI") %>%
      arrange(desc(record_intensity)) %>% 
      dplyr::select(key) %>%
      distinct() %>% dplyr::slice(1:2)
  ) %>%
  mutate(link=lapply(key,get_loinc_ref))


A total of `r n_distinct(lab_summ$key)` LOINC identifiable labs are eligible (NI may present), among which `r nrow(lab_summ %>% filter(at_admission==1) %>% dplyr::select(key) %>% distinct())` are collected at the day of admission. Figure 1 shows the data density and intensity of labs concepts, which can help identify common labs (e.g. the common labs for this study cohort are [`r lab_report$key[1]`], [`r lab_report$key[2]`],[`r lab_report$key[3]`],[`r lab_report$key[4]`],[`r lab_report$key[5]`],...), and labs with very high recording intensity (e.g. [`r lab_report$key[6]`], [`r lab_report$key[7]`]). 

*Note that only labs with coverage above 1% of the overall cohort are included in the plot, the complete distribution can be found in the accompanied excel file.* 

[`r lab_report$key[1]`]: `r lab_report$link[1]`
[`r lab_report$key[2]`]: `r lab_report$link[2]` 
[`r lab_report$key[3]`]: `r lab_report$link[3]`
[`r lab_report$key[4]`]: `r lab_report$link[4]`
[`r lab_report$key[5]`]: `r lab_report$link[5]`
[`r lab_report$key[6]`]: `r lab_report$link[6]`
[`r lab_report$key[7]`]: `r lab_report$link[7]`

***

##### Diagnosis

In [ ]:
## historical diagnosis
dx<-janitor::clean_names(read.csv(paste0(dataFolder,"AKI_DX",".csv")), case='all_cap') %>% 
dplyr::rename(ENCOUNTERID=ONSETS_ENCOUNTERID) %>%
  #attach CCS diagnosis grouping
  dplyr::mutate(DX_ICD=paste0("ICD",DX_TYPE,":",DX)) %>%
  left_join(readRDS("/home/hchan2/AKI/AKI_CDM/ref/ccs_icd_cw.rda") %>%
              select(-ccs_name),by=c("DX_ICD"="icd_w_type")) %>%
  distinct() %>% filter(!is.na(ccs_code)) %>%
  dplyr::rename(key=ccs_code, dsa=DAYS_SINCE_ADMIT) %>%
  dplyr::select(PATID,ENCOUNTERID,key,dsa) %>%
  distinct()

#collect summaries
dx_summ<-dx %>%
  group_by(key) %>%
  dplyr::summarize(record_cnt=n(),
                   pat_cnt=n_distinct(PATID),
                   enc_cnt=n_distinct(ENCOUNTERID),
                   min_history=min(dsa,na.rm=T),
                   mean_history=round(mean(dsa,na.rm=T)),
                   sd_history=round(sd(dsa,na.rm=T)),
                   median_history=round(median(dsa,na.rm=T)),
                   max_history=max(dsa,na.rm=T)) %>%
  ungroup %>%
  #HIPAA, low counts masking
  mutate(pat_cnt=ifelse(as.numeric(pat_cnt)<11,"<11",pat_cnt),
         enc_cnt=ifelse(as.numeric(enc_cnt)<11,"<11",enc_cnt),
         record_cnt=ifelse(as.numeric(record_cnt)<11,"<11",record_cnt)) %>%
  arrange(key)

#save
#saveRDS(dx,file=paste0(targetFolder,"DX","_POSIX",".rda"))
#saveRDS(dx_summ,file=paste0(targetFolder,"dx_summ","_POSIX",".rda"))

#clean up
rm(dx); gc()

In [ ]:
#historical diagnosis
dx_temp<-dx_summ %>% 
  dplyr::select(key,enc_cnt,mean_history) %>%
  filter(enc_cnt!="<11") %>%
  mutate(enc_cnt=as.numeric(enc_cnt),
         mean_history=as.numeric(mean_history)) %>%
  filter(enc_cnt>=enc_tot*0.01) %>%
  mutate(label=ifelse(dense_rank(-enc_cnt)<=10,key,""))

ggplot(dx_temp,aes(x=mean_history,y=enc_cnt,label=label))+
  geom_point()+geom_text_repel()+
  scale_y_continuous(sec.axis = sec_axis(trans= ~./enc_tot,
                                         name = 'Percentage'))+
  labs(x="Mean History of Diagnoses (Days)",
       y="Encounter Counts",
       title="Figure 2 - Data Density vs. Recency (CCS)")

dx_report<-dx_temp %>% 
  arrange(desc(enc_cnt)) %>%
  dplyr::slice(1:6) %>%
  dplyr::select(key) %>%
  left_join(readRDS("/home/hchan2/AKI/AKI_CDM/ref/ccs_ref.rda") %>% 
              filter(type=="dx"),
            by=c("key"="ccs_code")) %>%
  dplyr::select(key,ccs_name)

A Total of `r n_distinct(dx_summ$key)` distinct CCS-grouped diagnoses has been assigned to patients before the encounter of interest. Figure 2 gives an overview of average history of patients' diagnosis prior to tne encounter of interest as well as the highly frequent historical diagnoses(e.g. `r dx_report$key[1]`(`r dx_report$ccs_name[1]`), `r dx_report$key[2]`(`r dx_report$ccs_name[2]`), `r dx_report$key[3]`(`r dx_report$ccs_name[3]`), `r dx_report$key[4]`(`r dx_report$ccs_name[4]`), `r dx_report$key[5]`(`r dx_report$ccs_name[5]`), `r dx_report$key[6]`(`r dx_report$ccs_name[6]`)). 

*Note that only ccs diagnosis codes with coverage above 1% of the overall cohort are included in the plot, the complete distribution can be found in the accompanied excel file.*

***

##### Procedure

In [ ]:
## procedure
px<-janitor::clean_names(read.csv(paste0(dataFolder,"AKI_PX",".csv")), case='all_cap') %>% 
dplyr::rename(ENCOUNTERID=ONSETS_ENCOUNTERID) %>%
  dplyr::mutate(PX=paste0(PX_TYPE,":",PX)) %>%
  dplyr::rename(key=PX,dsa=DAYS_SINCE_ADMIT) %>%
  dplyr::select(PATID,ENCOUNTERID,key,dsa) %>%
  distinct() %>%
  mutate(dsa_grp=case_when(dsa < 0 ~ "0>",
                           dsa >=0 & dsa < 1 ~ "1",
                           dsa >=1 & dsa < 2 ~ "2",
                           dsa >=2 & dsa < 3 ~ "3",
                           dsa >=3 & dsa < 4 ~ "4",
                           dsa >=4 & dsa < 5 ~ "5",
                           dsa >=5 & dsa < 6 ~ "6",
                           dsa >=6 & dsa < 7 ~ "7",
                           dsa >=7 ~ "7<"))

px_summ<-px %>%
  group_by(key,dsa_grp) %>%
  dplyr::summarize(record_cnt=n(),
                   pat_cnt=n_distinct(PATID),
                   enc_cnt=n_distinct(ENCOUNTERID)) %>%
  ungroup %>%
  #HIPAA, low counts masking
  mutate(pat_cnt=ifelse(as.numeric(pat_cnt)<11,"<11",pat_cnt),
         enc_cnt=ifelse(as.numeric(enc_cnt)<11,"<11",enc_cnt),
         record_cnt=ifelse(as.numeric(record_cnt)<11,"<11",record_cnt)) %>%
  arrange(key,dsa_grp)

#save
#saveRDS(px,file=paste0(targetFolder,"PX","_POSIX",".rda"))
#saveRDS(px_summ,file=paste0(targetFolder,"px_summ","_POSIX",".rda"))

#clean up
rm(px); gc()

In [ ]:
px_temp<-px_summ %>% 
  dplyr::select(key,dsa_grp,enc_cnt) %>% 
  filter(enc_cnt!="<11") %>%
  mutate(enc_cnt=as.numeric(enc_cnt)) %>%
  filter(enc_cnt>=enc_tot*0.001) %>%
  arrange(desc(enc_cnt)) %>%
  mutate(label=ifelse(dense_rank(-enc_cnt)<=10,key,""))

ggplot(px_temp,aes(x=dsa_grp,y=enc_cnt,label=label))+
  geom_point()+geom_text_repel()+
  scale_y_continuous(sec.axis = sec_axis(trans= ~./enc_tot,
                                         name = 'Percentage'))+
  labs(x="Days since Admission",
       y="Encounter Counts",
       title="Figure 3 - Procedure Density over Time")

px_report<-px_temp %>%
  arrange(desc(enc_cnt)) %>%
  dplyr::select(key) %>%
  distinct() %>% dplyr::slice(1:5) %>%
  # mutate(link=lapply(key,google_code)) #uncomment it if google_code doesn't cause error
  # mutate(link="https://coder.aapc.com/cpt-codes/") #comment it if google_code doesn't cause error
  mutate(link=paste0("https://www.google.com/search?q=",ifelse(str_detect(key, '^CH'), gsub(':','+aapc.com+',key), gsub(':','+icd+',key))))

A Total of `r n_distinct(px_summ$key)` distinct total procedures codes have been assigned to patients during the encounter of interest. Figure3 gives an overview of procedures that patients had recieved before or at the encounters of interest. It can help identify the common procedures or typical occuring times of precedures (e.g. [`r px_report$key[1]`], [`r px_report$key[2]`],[`r px_report$key[3]`],[`r px_report$key[4]`],[`r px_report$key[5]`],...). 

*Note that only procedure codess with coverage above 0.1% of the overall cohort are included in the plot, the complete distribution can be found in the accompanied excel file.*

[`r px_report$key[1]`]: `r px_report$link[1]`
[`r px_report$key[2]`]: `r px_report$link[2]` 
[`r px_report$key[3]`]: `r px_report$link[3]`
[`r px_report$key[4]`]: `r px_report$link[4]`
[`r px_report$key[5]`]: `r px_report$link[5]`

***

In [ ]:
med_eval<-file.exists(paste0(dataFolder,"AKI_AMED",".csv"))
warningMessage<-""
if (med_eval){
    med<-janitor::clean_names(read.csv(paste0(dataFolder,"AKI_AMED",".csv")), case='all_cap') %>% 
    dplyr::rename(ENCOUNTERID=ONSETS_ENCOUNTERID) %>%    
#		med_eval<-(!(all(is.na(med$MEDADMIN_STOP_DATE_TIME))))
    if (all(is.na(med$MEDADMIN_STOP_DATE_TIME))){
      warningMessage<-"WARNING: MEDADMIN_STOP_DATE_TIME do not exists, all exposure is assumed to be 0 in following graph."
      med$MEDADMIN_STOP_DATE_TIME<-med$MEDADMIN_START_DATE_TIME
    } else if (anyNA(med$MEDADMIN_STOP_DATE_TIME)) {
      warningMessage<-"WARNING: some MEDADMIN_STOP_DATE_TIME do not exists, those exposure is assumed to be 0 in following graph."
      med %<>% dplyr::mutate(MEDADMIN_STOP_DATE_TIME=coalesce(MEDADMIN_STOP_DATE_TIME, MEDADMIN_START_DATE_TIME))
    }
}
if (!(med_eval)){
    description2<-"Med Admin Table is missing or not complete"
    med_summ<-data.frame()
}

##### Medications
`r warningMessage`

In [ ]:
## medication
med%<>%
#  dplyr::mutate(RX_EXPOS=0) %>%
  dplyr::mutate(RX_EXPOS=round(pmax(as.numeric(difftime(MEDADMIN_START_DATE_TIME,MEDADMIN_STOP_DATE_TIME,units="days")),1))) %>%
  dplyr::rename(sdsa=DAYS_SINCE_ADMIT) %>%
  dplyr::select(PATID,ENCOUNTERID,MEDADMIN_CODE,MEDADMIN_TYPE,MEDADMIN_ROUTE,RX_EXPOS,sdsa) %>%
  mutate(RX_QUANTITY_DAILY=1) %>%
  unite("key",c("MEDADMIN_CODE","MEDADMIN_TYPE","MEDADMIN_ROUTE"),sep=":")
  
#re-calculate medication exposure
chunk_num<-1
enc_chunk<-med %>% dplyr::select(ENCOUNTERID) %>% distinct() %>%
  mutate(chunk_id=sample(1:chunk_num,n(),replace=T))

med2<-c()
for(i in 1:chunk_num){
  start_i<-Sys.time()

  #--subset ith chunk
  med_sub<-med %>%
    semi_join(enc_chunk %>% filter(chunk_id==i),by="ENCOUNTERID")

  #--collect single-day exposure
#  med_sub2<-med_sub %>% filter(RX_EXPOS<=1) %>%
  med_sub2<-med_sub %>%
    dplyr::mutate(dsa=sdsa,value=RX_QUANTITY_DAILY) %>%
    dplyr::select(ENCOUNTERID,key,value,dsa)

  #--for multi-day exposed med, converted to daily exposure
  # med_expand<-med_sub[rep(row.names(med_sub),(med_sub$RX_EXPOS+1)),] %>%
  #   group_by(ENCOUNTERID,key,RX_QUANTITY_DAILY,sdsa) %>%
  #   dplyr::mutate(expos_daily=1:n()-1) %>%
  #   dplyr::summarize(dsa=paste0(sdsa+expos_daily,collapse=",")) %>%
  #   ungroup %>% dplyr::rename(value=RX_QUANTITY_DAILY) %>%
  #   dplyr::select(ENCOUNTERID,key,value,dsa) %>%
  #   mutate(dsa=strsplit(dsa,",")) %>%
  #   unnest(dsa) %>%
  #   mutate(dsa=as.numeric(dsa))

  #--merge overlapped precribing intervals (pick the higher exposure)
#  med_sub2 %<>% bind_rows(med_expand) %>%
  med_sub2 %<>%
    group_by(ENCOUNTERID,key,dsa) %>%
    dplyr::summarize(value=max(value)) %>%
    ungroup

  #--identify non-overlapped exposure episodes and determines the real sdsa
  med_sub2 %<>%
    group_by(ENCOUNTERID,key) %>%
    dplyr::mutate(dsa_lag=lag(dsa,n=1L)) %>%
    ungroup %>%
    mutate(sdsa=ifelse(is.na(dsa_lag)|dsa > dsa_lag+1,dsa,NA)) %>%
    fill(sdsa,.direction="down")

  med_sub2 %<>%
    group_by(ENCOUNTERID,key,sdsa) %>%
    dplyr::summarize(RX_EXPOS=pmax(1,sum(value,na.rm=T)),
                     value=paste0(value,collapse=","), #expanded daily exposure
                     dsa=paste0(dsa,collapse=",")) %>%  #expanded dsa for daily exposure
    ungroup

  med2 %<>% bind_rows(med_sub2)
}
med<-med2

#collect summaries
med_summ<-med %>% 
  dplyr::select(ENCOUNTERID,key,sdsa,RX_EXPOS) %>%
  mutate(dsa_grp=case_when(sdsa < 0 ~ "0>",
                           sdsa >=0 & sdsa < 1 ~ "1",
                           sdsa >=1 & sdsa < 2 ~ "2",
                           sdsa >=2 & sdsa < 3 ~ "3",
                           sdsa >=3 & sdsa < 4 ~ "4",
                           sdsa >=4 & sdsa < 5 ~ "5",
                           sdsa >=5 & sdsa < 6 ~ "6",
                           sdsa >=6 & sdsa < 7 ~ "7",
                           sdsa >=7 ~ "7<")) %>%
  group_by(key,dsa_grp) %>%
  dplyr::summarize(record_cnt=n(),
                   enc_cnt=n_distinct(ENCOUNTERID),
                   min_expos=min(RX_EXPOS,na.rm=T),
                   mean_expos=round(mean(RX_EXPOS,na.rm=T)),
                   sd_expos=round(sd(RX_EXPOS,na.rm=T)),
                   median_expos=round(median(RX_EXPOS,na.rm=T)),
                   max_expos=max(RX_EXPOS,na.rm=T)) %>%
  ungroup %>%
  #HIPPA, low counts masking
  mutate(enc_cnt=ifelse(as.numeric(enc_cnt)<11,"<11",as.character(enc_cnt)),
         record_cnt=ifelse(as.numeric(record_cnt)<11,"<11",as.character(record_cnt)),
         sd_expos=ifelse(is.na(sd_expos),0,sd_expos)) %>%
  dplyr::mutate(cov_expos=round(sd_expos/mean_expos,1)) %>%
  gather(summ,summ_val,-key,-dsa_grp) %>%
  spread(dsa_grp,summ_val) %>%
  arrange(key,summ)

med_density<-n_distinct(med$ENCOUNTERID)

#save
#saveRDS(med,file=paste0(targetFolder,"MED","_POSIX",".rda"))
#saveRDS(med_summ,file=paste0(targetFolder,"med_summ","_POSIX",".rda"))

#clean up
rm(med); gc()

In [ ]:
med_temp<-med_summ %>% 
  filter(summ %in% c("enc_cnt","median_expos")) %>% 
#  filter(summ %in% c("enc_cnt")) %>% 
  gather(dsa_grp,summ_val,-summ,-key) %>%
  filter(!is.na(summ_val) & (summ_val!="<11")) %>%
  mutate(summ_val=as.numeric(summ_val)) %>%
  spread(summ,summ_val) %>%
#  filter(!is.na(median_expos) & enc_cnt>=enc_tot*0.001) %>%
#  filter(enc_cnt>=enc_tot*0.001) %>%
  filter(!is.na(enc_cnt)) %>%
#  arrange(median_expos) %>%
  group_by(dsa_grp) %>%
  # mark the high-frequent meds
  dplyr::mutate(label=ifelse(rank(-enc_cnt,ties.method="random")<=3,key,"")) %>%
  ungroup
  # mark the long-exposed meds
#  dplyr::mutate(label=ifelse(label!="",label,
#                             ifelse(rank(-median_expos,ties.method="random")<=5,key,"")))


if(nrow(med_temp)>0){
#  overall_medexpos<-median(med_temp$median_expos,na.rm=T)
  # p1<-ggplot(med_temp,aes(x=dsa_grp,y=enc_cnt,color=median_expos,label=label)) +
  #   geom_point() + geom_text_repel()+
  #   scale_y_continuous(sec.axis = sec_axis(trans= ~./enc_tot,
  #                                          name = 'Percentage'))+
  #   scale_color_gradient2(low = "green",mid="blue",high ="red",
  #                         midpoint = overall_medexpos)+
  #   labs(x="Start Date",y="Encounter Counts",color="Median Exposure (days)",
  #        title="Figure 4 - Medication Exposure Summaries")

  p1<-ggplot(med_temp,aes(x=dsa_grp,y=enc_cnt,label=label)) +
    geom_point() + geom_text_repel()+
    scale_y_continuous(sec.axis = sec_axis(trans= ~./enc_tot,
                                           name = 'Percentage'))+
    labs(x="Start Date",y="Encounter Counts",
         title="Figure 4 - Medication Summaries")
    
  print(p1) #need print() to demonstrate the plot in top-layer expression(e.g. ifelse)
  
  med_report<-med_temp %>%
    separate(key,c("key2","type","route"),":",extra="merge",fill="right") %>%
    mutate(rx_name=trimws(key2),
           type=trimws(type))
#  med_reportRX <- med_report %>%
#    filter(type=='RX') %>%
#    mutate(rx_name=lapply(key2,get_rxcui_nm))
#  med_reportND <- med_report %>%
#    filter(type!='RX') %>%
#    mutate(rx_name=lapply(key2,get_ndc_nm))
#  med_report<-rbind(med_reportRX,med_reportND)
  med_report %<>% mutate(rx_name=key2)
  med_report %<>%
    arrange(desc(enc_cnt)) %>%
    dplyr::select(key2,type,rx_name,enc_cnt) %>% distinct() %>%
    mutate(rx_name=trimws(rx_name)) %>%
    group_by(rx_name) %>%
    dplyr::summarise(key=paste(unique(key2),collapse = ","),enc_cnt=sum(enc_cnt)) %>%
    arrange(desc(enc_cnt)) %>%
    ungroup %>% filter(!is.null(rx_name)&!rx_name %in% c("NULL"))
  
  freq_med<-c()
  for(k in 1:nrow(med_report)){
    freq_med<-c(freq_med,paste0(get_rxcui_nm(med_report$rx_name[k]),"(",med_report$key[k],")")) 
  }
  
  med_report<-med_temp %>%
    separate(key,c("key2","type","route"),":",extra="merge",fill="right") %>%
    mutate(rx_name=trimws(key2),
           type=trimws(type))
#  med_reportRX <- med_report %>%
#    filter(type=='RX') %>%
#    mutate(rx_name=lapply(key2,get_rxcui_nm))
#  med_reportND <- med_report %>%
#    filter(type!='RX') %>%
#    mutate(rx_name=lapply(key2,get_ndc_nm))
#  med_report<-rbind(med_reportRX,med_reportND)
  med_report %<>% mutate(rx_name=key2)
  med_report %<>%
    arrange(desc(median_expos)) %>%
    dplyr::select(key2,type,rx_name,median_expos) %>% distinct() %>%
    mutate(rx_name=trimws(rx_name)) %>%
    group_by(rx_name) %>%
    dplyr::summarise(key=paste(unique(key2),collapse = ",")) %>%
    ungroup %>% filter(!is.null(rx_name)&!rx_name %in% c("NULL"))

  intens_med<-c()
  for(k in 1:nrow(med_report)){
    intens_med<-c(intens_med,paste0(get_rxcui_nm(med_report$rx_name[k]),"(",med_report$key[k],")")) 
  }
  
  description<-paste0("Figure4 demonstrates average exposures for drug starting at X days since admission. 
                      It helps identify typical medciations administered during the course of stay. 
                      (e.g. the typical medications identified are ",paste(freq_med[1:3],collapse=","),
                      "; while drugs such as ",paste(intens_med[1:3],collapse=","), 
                      " are used with a relative longer exposure than the others).")
}else{
  description<-"Medication exposure are too low as no medication identifier has a coverage of more than 0.1% of the study population."
}

description2<-paste0("A Total of ", n_distinct(med_summ$key), " distinct medication concepts (RXNORM and/or NDC) are discovered for the cohort and the overall medication (any) exposure for this cohort is ", paste0(round(med_density/enc_tot,2)*100,"%"), ". ", description) 

`r description2`

*Note that only rxnorms with coverage above 0.1% of the overall cohort are included in the plot, the complete distribution can be found in the accompanied excel file.*


In [ ]:
final_out<-list(Table1=consort_tbl,
                Table2=tbl1_summ,
                Table3=demo_nice_tbl,
                Table4=vital_summ,
                Table5=vital_smoke_summ,
                Table6=lab_summ,
                Table7=dx_summ,
                Table8=px_summ,
                Table9=med_summ)
write.xlsx(final_out,file=paste0(targetFolder,"AKI_CDM_EXT_VALID_p1_QA_TBL.xlsx"))


# ggplot sometimes create some unwanted empty .pdf file and want to clean it up
if(file.exists("./Rplots.pdf")){
  file.remove("./Rplots.pdf")
}

rm(list=ls())
gc()

***

